# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.847768065569                   -0.70    4.5   83.2ms
  2   -7.852391886513       -2.33       -1.53    1.0   17.1ms
  3   -7.852609711015       -3.66       -2.55    1.2   17.4ms
  4   -7.852645462427       -4.45       -2.78    2.5   22.2ms
  5   -7.852646059784       -6.22       -2.86    1.2   17.7ms
  6   -7.852646667721       -6.22       -3.85    1.0   16.9ms
  7   -7.852646685003       -7.76       -4.66    1.5   18.6ms
  8   -7.852646686687       -8.77       -5.02    1.8   20.0ms
  9   -7.852646686715      -10.55       -5.26    1.0   17.1ms
 10   -7.852646686727      -10.89       -5.63    1.0   17.7ms
 11   -7.852646686730      -11.64       -6.37    1.2   17.9ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.847732570054                   -0.70           4.8    418ms
  2   -7.852560326211       -2.32       -1.62   0.80    2.0    2.21s
  3   -7.852640980890       -4.09       -2.70   0.80    1.0    223ms
  4   -7.852646535848       -5.26       -3.40   0.80    2.0   19.6ms
  5   -7.852646679098       -6.84       -4.45   0.80    1.5   17.4ms
  6   -7.852646686610       -8.12       -4.83   0.80    2.0   20.2ms
  7   -7.852646686725       -9.94       -5.49   0.80    1.0   15.9ms
  8   -7.852646686730      -11.37       -6.57   0.80    1.8   18.1ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +1.302867381357                   -1.02    4.15s
  2   -1.473996605246        0.44       -0.70    104ms
  3   -3.877556968770        0.38       -0.44   32.7ms
  4   -5.137589003036        0.10       -0.58   32.7ms
  5   -6.817505478699        0.23       -0.75   32.7ms
  6   -6.958464135979       -0.85       -1.47   24.6ms
  7   -7.545286211606       -0.23       -1.58   25.1ms
  8   -7.691670519929       -0.83       -1.84   25.1ms
  9   -7.768680554181       -1.11       -1.85   24.9ms
 10   -7.800840892725       -1.49       -2.01   59.3ms
 11   -7.816952161262       -1.79       -2.20   25.2ms
 12   -7.836111721766       -1.72       -2.57   24.9ms
 13   -7.848584371312       -1.90       -2.58   24.7ms
 14   -7.851093983229       -2.60       -3.16   24.8ms
 15   -7.851829891347       -3.13       -3.41   24.8ms
 16   -7.852234795909       -3.39       -3.61   24.8ms
 17   -7.85

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.847773973845                   -0.70    4.8   30.4ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645851432                   -1.63    13.4s
  2   -7.852646686730       -6.08       -3.69    3.51s
  3   -7.852646686730      -13.22       -7.20    124ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 5.220810422208978e-7
|ρ_newton - ρ_scfv| = 2.862807611357331e-7
|ρ_newton - ρ_dm|   = 3.422723810254697e-10
